In [1]:
import os

Variaveis globais necessárias para funçoes

In [2]:
datasetTxt = "Datasets/Dataset-ClassTT_01_tiny_File.txt"


Parse do dataset de horários, retornando um dicionário com todas as seções.

    Retorna:
        dict: Dicionário com as seguintes chaves:
            - 'head': dict com configurações gerais
            - 'cc': dict {turma: [cursos]}
            - 'olw': list de cursos com apenas 1 aula/semana
            - 'dsd': dict {professor: [cursos]}
            - 'tr': dict {professor: [slots indisponíveis]}
            - 'rr': dict {curso: sala}
            - 'oc': dict {curso: índice_aula_online}

In [3]:
# Cell 3: Importações e funções auxiliares
from constraint import Problem
import itertools

def slot_day(slot):
    """Converte slot (1-20) para dia da semana (0=Segunda, 1=Terça, 2=Quarta, 3=Quinta, 4=Sexta)"""
    return (slot - 1) // 4

def slot_time(slot):
    """Converte slot (1-20) para período do dia (0=manhã1, 1=manhã2, 2=tarde1, 3=tarde2)"""
    return (slot - 1) % 4

def day_name(day_index):
    """Retorna o nome do dia"""
    days = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
    return days[day_index]

def time_name(time_index):
    """Retorna o nome do período"""
    times = ["9h-11h", "11h-13h", "14h-16h", "16h-18h"]
    return times[time_index]

def are_consecutive(slot1, slot2):
    """Verifica se dois slots são consecutivos no mesmo dia"""
    return slot_day(slot1) == slot_day(slot2) and abs(slot_time(slot1) - slot_time(slot2)) == 1

print("Funções auxiliares definidas")

Funções auxiliares definidas


In [4]:
# Cell 2: Parser do dataset (seu código)
def parse_dataset(datasetTxt):
    lines = datasetTxt.strip().split('\n')
    data = {
        'head': {},
        'cc': {},
        'olw': [],
        'dsd': {},
        'tr': {},
        'rr': {},
        'oc': {}
    }
    current_section = None

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('#'):
            section_name = line.split()[0][1:]
            current_section = section_name
            continue
        if line.startswith('—') or line.startswith('--'):
            continue
        if current_section == 'cc':
            parts = line.split()
            if parts:
                data['cc'][parts[0]] = parts[1:]
        elif current_section == 'olw':
            if line.strip():
                data['olw'].append(line.strip())
        elif current_section == 'dsd':
            parts = line.split()
            if parts:
                data['dsd'][parts[0]] = parts[1:]
        elif current_section == 'tr':
            parts = line.split()
            if parts:
                data['tr'][parts[0]] = [int(s) for s in parts[1:]]
        elif current_section == 'rr':
            parts = line.split()
            if len(parts) >= 2:
                data['rr'][parts[0]] = parts[1]
        elif current_section == 'oc':
            parts = line.split()
            if len(parts) >= 2:
                data['oc'][parts[0]] = int(parts[1])
    return data

def load_dataset_from_file(datasetTxt):
    try:
        with open(datasetTxt, 'r', encoding='utf-8') as f:
            dataset_text = f.read()
        return parse_dataset(dataset_text)
    except FileNotFoundError:
        print(f"Erro: Ficheiro '{datasetTxt}' não encontrado!")
        return None
    except Exception as e:
        print(f"Erro ao ler ficheiro: {e}")
        return None

print("✅ Parser do dataset definido")

✅ Parser do dataset definido


## Testes

In [5]:
# Cell 3: Carregar dataset
datasetTxt = "Datasets/Dataset-ClassTT_01_tiny_File.txt"
data = load_dataset_from_file(datasetTxt)

if data is None:
    print("❌ Falha ao carregar o dataset!")
else:
    print("✅ Dataset carregado com sucesso!\n")
    print("📊 CONTEÚDO PARSEADO:")
    print("CC (Turmas):", data['cc'])
    print("DSD (Professores):", data['dsd'])
    print("TR (Indisponibilidades):", data['tr'])
    print("RR (Salas):", data['rr'])
    print("OC (Online):", data['oc'])
    print("OLW (1 aula/semana):", data['olw'])

✅ Dataset carregado com sucesso!

📊 CONTEÚDO PARSEADO:
CC (Turmas): {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'], 't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'], 't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']}
DSD (Professores): {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}
TR (Indisponibilidades): {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}
RR (Salas): {'UC14': 'Lab01', 'UC22': 'Lab01'}
OC (Online): {'UC21': 2, 'UC31': 2}
OLW (1 aula/semana): []


In [6]:
!pip install python-constraint

In [7]:
# Cell 4: Configuração do problema CSP - Variáveis
print("🔄 Configurando problema CSP...")

problem = Problem()
ALL_SLOTS = list(range(1, 21))

# 1. CRIAR VARIÁVEIS
course_vars = {}
variables = []
total_vars = 0

for turma, courses in data['cc'].items():
    for course in courses:
        # Verificar se é curso com apenas 1 aula por semana (online ou OLW)
        if course in data.get('oc', {}) or course in data.get('olw', []):
            course_vars[course] = [f"{course}_1"]
            total_vars += 1
        else:
            course_vars[course] = [f"{course}_1", f"{course}_2"]
            total_vars += 2
        
        # Adicionar variáveis ao problema
        for var_name in course_vars[course]:
            variables.append(var_name)
            problem.addVariable(var_name, ALL_SLOTS)

print(f"✅ Criadas {total_vars} variáveis para {len(course_vars)} cursos")

🔄 Configurando problema CSP...
✅ Criadas 28 variáveis para 15 cursos


In [8]:
# Cell 5: Restrições Obrigatórias (Hard Constraints)
print("🔒 Aplicando restrições obrigatórias...")

# 1. Aulas do mesmo curso em blocos diferentes
constraints_added = 0
for course, varslist in course_vars.items():
    if len(varslist) == 2:
        problem.addConstraint(lambda a, b: a != b, tuple(varslist))
        constraints_added += 1
print(f"✅ {constraints_added} restrições de aulas diferentes")

# 2. Indisponibilidades dos professores
constraints_added = 0
for teacher, unavailable_slots in data['tr'].items():
    if teacher in data['dsd']:
        teacher_courses = data['dsd'][teacher]
        for course in teacher_courses:
            if course in course_vars:
                for var_name in course_vars[course]:
                    problem.addConstraint(lambda slot, us=unavailable_slots: slot not in us, [var_name])
                    constraints_added += 1
print(f"✅ {constraints_added} restrições de indisponibilidade")

# 3. Professor não pode ter duas aulas ao mesmo tempo
constraints_added = 0
for teacher, courses in data['dsd'].items():
    t_vars = []
    for course in courses:
        if course in course_vars:
            t_vars.extend(course_vars[course])
    
    if len(t_vars) >= 2:
        problem.addConstraint(lambda *vals: len(set(vals)) == len(vals), t_vars)
        constraints_added += 1
print(f"✅ {constraints_added} restrições de conflito de professores")

# 4. Turma não pode ter aulas no mesmo bloco
constraints_added = 0
for turma, courses in data['cc'].items():
    t_vars = []
    for course in courses:
        if course in course_vars:
            t_vars.extend(course_vars[course])
    
    if len(t_vars) >= 2:
        problem.addConstraint(lambda *vals: len(set(vals)) == len(vals), t_vars)
        constraints_added += 1
print(f"✅ {constraints_added} restrições de conflito de turmas")

# 5. Máx. 3 aulas por dia por turma
constraints_added = 0
for turma, courses in data['cc'].items():
    t_vars = []
    for course in courses:
        if course in course_vars:
            t_vars.extend(course_vars[course])
    
    if t_vars:
        def at_most_3_per_day(*slots):
            day_counts = {}
            for slot in slots:
                day = slot_day(slot)
                day_counts[day] = day_counts.get(day, 0) + 1
                if day_counts[day] > 3:
                    return False
            return True
        
        problem.addConstraint(at_most_3_per_day, t_vars)
        constraints_added += 1
print(f"✅ {constraints_added} restrições de máximo 3 aulas por dia")

# 6. Salas fixas
constraints_added = 0
room_to_courses = {}
for course, room in data.get('rr', {}).items():
    room_to_courses.setdefault(room, []).append(course)

for room, courses in room_to_courses.items():
    r_vars = []
    for course in courses:
        if course in course_vars:
            r_vars.extend(course_vars[course])
    
    if len(r_vars) >= 2:
        problem.addConstraint(lambda *vals: len(set(vals)) == len(vals), r_vars)
        constraints_added += 1
print(f"✅ {constraints_added} restrições de salas fixas")

# 7. Aulas online - máximo 3 por dia
online_vars = []
for course, lesson_index in data.get('oc', {}).items():
    var_name = f"{course}_{lesson_index}"
    if var_name in variables:
        online_vars.append(var_name)

if online_vars:
    def online_max_3_per_day(*slots):
        day_counts = {}
        for slot in slots:
            day = slot_day(slot)
            day_counts[day] = day_counts.get(day, 0) + 1
            if day_counts[day] > 3:
                return False
        return True
    
    problem.addConstraint(online_max_3_per_day, online_vars)
    print(f"✅ 1 restrição de máximo 3 aulas online por dia")
else:
    print("✅ Nenhuma restrição de aulas online aplicada")

print("🎯 Todas as restrições obrigatórias aplicadas")

🔒 Aplicando restrições obrigatórias...
✅ 13 restrições de aulas diferentes
✅ 22 restrições de indisponibilidade
✅ 4 restrições de conflito de professores
✅ 3 restrições de conflito de turmas
✅ 3 restrições de máximo 3 aulas por dia
✅ 1 restrições de salas fixas
✅ Nenhuma restrição de aulas online aplicada
🎯 Todas as restrições obrigatórias aplicadas


In [9]:
# Cell 6: Restrições Preferenciais (Soft Constraints)
print("💡 Aplicando restrições preferenciais...")

# 1. Aulas da mesma UC em dias distintos (quando possível)
constraints_added = 0
for course, varslist in course_vars.items():
    if len(varslist) == 2:
        problem.addConstraint(lambda a, b: slot_day(a) != slot_day(b), tuple(varslist))
        constraints_added += 1
print(f"✅ {constraints_added} restrições de dias distintos")

# 2. Preferência por 4 dias de aulas por turma
constraints_added = 0
for turma, courses in data['cc'].items():
    t_vars = []
    for course in courses:
        if course in course_vars:
            t_vars.extend(course_vars[course])
    
    if t_vars:
        def prefer_4_days(*slots):
            days_used = len(set(slot_day(s) for s in slots))
            # Penalizar se menos de 4 dias (mas não impossibilitar)
            return days_used >= 3  # Pelo menos 3 dias
            
        problem.addConstraint(prefer_4_days, t_vars)
        constraints_added += 1
print(f"✅ {constraints_added} restrições de preferência por 4 dias")

# 3. Aulas consecutivas no mesmo dia (quando possível)
constraints_added = 0
for course, varslist in course_vars.items():
    if len(varslist) == 2:
        def prefer_consecutive(a, b):
            if slot_day(a) == slot_day(b):
                return are_consecutive(a, b)
            return True  # Se estão em dias diferentes, não há problema
            
        problem.addConstraint(prefer_consecutive, tuple(varslist))
        constraints_added += 1
print(f"✅ {constraints_added} restrições de aulas consecutivas")

print("🎯 Todas as restrições preferenciais aplicadas")

💡 Aplicando restrições preferenciais...
✅ 13 restrições de dias distintos
✅ 3 restrições de preferência por 4 dias
✅ 13 restrições de aulas consecutivas
🎯 Todas as restrições preferenciais aplicadas


In [ ]:
# Cell 7: Resolver o problema
print("🔍 A procurar solução...")

solution = problem.getSolution()

if solution:
    print("🎉 SOLUÇÃO ENCONTRADA!")
    print("=" * 60)
    
    # Organizar solução por turma
    for turma, courses in data['cc'].items():
        print(f"\n📅 HORÁRIO DA TURMA {turma}:")
        print("-" * 40)
        
        # Coletar todas as aulas da turma
        turma_aulas = []
        for course in courses:
            if course in course_vars:
                for var_name in course_vars[course]:
                    if var_name in solution:
                        slot = solution[var_name]
                        day = day_name(slot_day(slot))
                        time = time_name(slot_time(slot))
                        turma_aulas.append((slot, day, time, var_name, course))
        
        # Ordenar por slot
        turma_aulas.sort(key=lambda x: x[0])
        
        # Imprimir horário organizado
        for slot, day, time, var_name, course in turma_aulas:
            online_flag = " 💻" if course in data.get('oc', {}) else ""
            room_info = f" ({data['rr'][course]})" if course in data.get('rr', {}) else ""
            print(f"  {day} {time}: {course}{online_flag}{room_info}")
    
else:
    print("❌ NENHUMA SOLUÇÃO ENCONTRADA!")

🔍 A procurar solução...
